## Libraries to be installed

In [1]:
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40716 sha256=e69e56df61581779d0817294141a8d581392e12e33dc9de507dd0ae25604b40d
  Stored in directory: c:\users\nawaf\appdata\local\pip\cache\wheels\d9\5f\46\54a2bdb4bcb07d3faba4463d2884865705914cc72a7b8bb5f0
Successfully built googlemaps


In [239]:
!pip install python_tsp

In [240]:
!pip install gmaps

## Importing all needed libraries

In [9]:
import googlemaps
from datetime import datetime
import pandas as pd
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_simulated_annealing
import gmaps

## API Key (don't leak please)

In [1]:
API_key = "YOUR GOOGLE API KEY GOES HERE"

### Run to optimize bus route U1

In [22]:
# Initializing Google Maps Client
gmaps_client = googlemaps.Client(key = API_key)

# Reading U1 bus stop dataset into dataframe df
df = pd.read_csv('U1_bus_stops.csv')
df['distance_text'] = np.nan
df['duration_text'] = np.nan

# Setting start_time for optimized route
start_time = datetime.now()
# start_time = datetime(2023, 2, 9, 15, 0, 0, 0)

# Setting bus stops to number of bus stops in route U1
bus_stops = 14

# df.head()

### Run to optimize bus route U2

In [4]:
# Initializing Google Maps Client
gmaps_client = googlemaps.Client(key = API_key)

# Reading U1 bus stop dataset into dataframe df
df = pd.read_csv('U2_bus_stops.csv')
df['distance_text'] = np.nan
df['duration_text'] = np.nan

# Setting start_time for optimized route
start_time = datetime.now()
# start_time = datetime(2023, 2, 9, 15, 0, 0, 0)

# Setting bus stops to number of bus stops in route U2
bus_stops = 19

# df.head()

## Creating Distance Variables

In [23]:
# Adding distance and travel_time information to the dataframe
for index, row in df.iterrows():
    if row['from_node'] != row['to_node']:
        direction_result = gmaps_client.directions(row['from_lat_long'], row['to_lat_long'], 
                                                   mode="driving", departure_time=start_time)
            
        df.loc[index, 'distance'] = direction_result[0]['legs'][0]['distance']['value']
        df.loc[index, 'travel_time'] = direction_result[0]['legs'][0]['duration']['value']
        df.loc[index, 'distance_text'] = direction_result[0]['legs'][0]['distance']['value']
        df.loc[index, 'duration_text'] = direction_result[0]['legs'][0]['duration']['value']
        
# df.head()

In [24]:
# Creating distance data dictionary containing distance from each stop to other stops
distance_dict = df.set_index(['from_node', 'to_node'])['distance'].to_dict()

# distance_dict

In [25]:
# Creating distance matrix from distance dictionary
distance = []

for from_node in df.from_node.unique().tolist():
    distance_1d = []
    for to_node in df.to_node.unique().tolist():
        distance_1d.append(distance_dict[from_node, to_node]/1000)
    distance.append(distance_1d)
    
# distance

In [26]:
# Creating duration data dictionary containing travel time from each stop to other stops
duration_dict = df.set_index(['from_node', 'to_node'])['travel_time'].to_dict()

# duration_dict

In [27]:
# Creating duration matrix from duration dictionary
duration = []

for from_node in df.from_node.unique().tolist():
    duration_1d = []
    for to_node in df.to_node.unique().tolist():
        duration_1d.append(duration_dict[from_node, to_node]/60)
    duration.append(duration_1d)
    
# duration

## Applying TSP Using the Distance and Duration Matrices

In [28]:
# Converting distance and duration matrices into arrays
distance_matrix = np.array(distance)
duration_matrix = np.array(duration)

In [29]:
# Using optimization algorithm on duration and distance matrices

# Permutation of stops to give optimized distance travelled
permutation, distance = solve_tsp_simulated_annealing(distance_matrix)

# Permutation of stops to give optimized time travelled
permutation, duration = solve_tsp_simulated_annealing(duration_matrix)

# permutation

In [30]:
# Mapping permutation indices to their corresponding labels
city_id_dict = df[['city_id', 'from_node']].drop_duplicates().set_index(['city_id'])['from_node'].to_dict()

route = []
for i in permutation:
    route.append(city_id_dict[i])
route.append(city_id_dict[0])

## Plotting the Optimal Route Output

In [31]:
# Creating latitude dictionary
lat_dict = df[['from_node', 'from_lat']].drop_duplicates().set_index(['from_node'])['from_lat'].to_dict()

# lat_dict

In [32]:
long_dict = df[['from_node', 'from_long']].drop_duplicates().set_index(['from_node'])['from_long'].to_dict()

# long_dict

In [33]:
# Printing the optimal route output to the screen
route_output = ""

for stop in route:
    route_output += " --> " + stop 
    
print(route_output)
print("\n")
print("The distance of the optimized route is", distance, "km")
print("The optimized route duration is", duration+(bus_stops*1.5), "minutes")

 --> Isa Town Terminal 2 --> Isa Town PID Stop --> Sh. Salman Highway-10 Jary AlSheikh Riffa --> Safreh Housing Riffa --> Bahrain University Main Gate 1 --> University of Bahrain - Sakhir 2 --> Bahrain University Main Gate/2 Zallaq --> Sh. Salman Highway-11 Riffa --> 16 December HWY-1 Isa Town --> Sh. Salman Highway-13 --> Sh. Abdulla School-1 Isa Town --> Educational Area - 2 Isa Town --> Bahrain University Isa Town --> Isa Town Gate-2 Isa Town --> Isa Town Terminal 2


The distance of the optimized route is 42.833 km
The optimized route duration is 83.35 minutes


In [34]:
# Generating the Google Maps figure with the optimal route mapped out
gmaps.configure(api_key = API_key)
fig = gmaps.figure()
for i in range(len(route)-1):
    from_node = route[i]
    to_node = route[i+1]
    
    layer = gmaps.directions.Directions((lat_dict[from_node], long_dict[from_node]),
                                       (lat_dict[to_node], long_dict[to_node]), mode="driving", departure_time=start_time)
    fig.add_layer(layer)

fig

Figure(layout=FigureLayout(height='420px'))